In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [2]:
feature = pd.read_csv('train_data.csv')
target = pd.read_csv('train_target.csv', header=None)

In [3]:
feature.head(3)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,26,student,single,high.school,no,no,no,telephone,jun,mon,901,1,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1
1,46,admin.,married,university.degree,no,yes,no,cellular,aug,tue,208,2,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1
2,49,blue-collar,married,basic.4y,unknown,yes,yes,telephone,jun,tue,131,5,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1


In [4]:
feature.shape

(27595, 20)

In [5]:
target.shape

(27595, 1)

удалим duration, тк он не нужен в обучении (см описание признаков):

In [6]:
feature.drop(['duration'], axis=1, inplace=True)

In [7]:
feature_categorial = feature[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']]

In [8]:
feature_categorial.shape[0]

27595

превратим категориальные признаки в бинарные:

In [9]:
feature_categorial = pd.get_dummies(feature[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']])
feature_categorial.head(3)

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [10]:
target.columns

Int64Index([0], dtype='int64')

In [11]:
feature.drop(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome'], axis=1, inplace=True)

In [12]:
feature = pd.concat((feature, feature_categorial), axis=1)
feature = pd.DataFrame(feature)
print (feature.shape)
print (feature.columns)

(27595, 62)
Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oc

In [13]:
feature.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_admin.,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,26,1,999,0,1.4,94.465,-41.8,4.961,5228.1,0,...,0,0,0,1,0,0,0,0,1,0
1,46,2,999,0,1.4,93.444,-36.1,4.963,5228.1,1,...,0,0,0,0,0,1,0,0,1,0
2,49,5,999,0,1.4,94.465,-41.8,4.864,5228.1,0,...,0,0,0,0,0,1,0,0,1,0
3,31,1,999,0,-2.9,92.469,-33.6,1.044,5076.2,0,...,0,0,0,0,0,1,0,0,1,0
4,42,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,0,...,0,0,0,1,0,0,0,0,1,0


IMBALANCED CLASSES

In [14]:
target[0].value_counts()

0    24481
1     3114
Name: 0, dtype: int64

In [15]:
X_train=feature
y_train=target

In [17]:
type(y_train)

pandas.core.frame.DataFrame

### XGBoost, fitting parameters with grid search and prediction

In [40]:
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV

C:\Users\Victoria\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Victoria\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [42]:
import xgboost as xgb

In [43]:
# xgb.XGBClassifier

In [44]:
y_train.shape[0]

27595

In [45]:
model = xgb.XGBClassifier(subsample=0.8, colsample_bytree=0.5)

In [46]:
model = xgb.XGBClassifier()
label_encoded_y = LabelEncoder().fit_transform(y_train)
max_depth = np.linspace(1,5,5,dtype=int)
n_estimators = np.linspace(120,550,10,dtype=int)
min_child_weight_range = np.linspace(1,5,5,dtype=int)

C:\Users\Victoria\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [69]:
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, min_child_weight = min_child_weight_range)
kfold = StratifiedKFold(label_encoded_y, n_folds=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(model, param_grid, scoring="roc_auc", n_jobs=-1, cv=kfold, verbose=1)
result = grid_search.fit(X_train, label_encoded_y)
# summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
means, stdevs = [], []
for params, mean_score, scores in result.grid_scores_:
    stdev = scores.std()
    means.append(mean_score)
    stdevs.append(stdev)
    print("%f (%f) with: %r" % (mean_score, stdev, params))
# plot results
scores = [x[1] for x in result.grid_scores_]
scores = numpy.array(scores).reshape(len(max_depth), len(n_estimators))
for i, value in enumerate(max_depth):
    pyplot.plot(n_estimators, scores[i], label='depth: ' + str(value))
pyplot.legend()
pyplot.xlabel('n_estimators')
pyplot.ylabel('auc roc')

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 30.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 79.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 178.6min
[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed: 180.2min finished


Best: 0.948645 using {'n_estimators': 275, 'max_depth': 4, 'min_child_weight': 2}
0.934320 (0.005450) with: {'n_estimators': 120, 'max_depth': 1, 'min_child_weight': 1}
0.935473 (0.005406) with: {'n_estimators': 151, 'max_depth': 1, 'min_child_weight': 1}
0.936281 (0.005243) with: {'n_estimators': 182, 'max_depth': 1, 'min_child_weight': 1}
0.936864 (0.005076) with: {'n_estimators': 213, 'max_depth': 1, 'min_child_weight': 1}
0.937210 (0.005000) with: {'n_estimators': 244, 'max_depth': 1, 'min_child_weight': 1}
0.937517 (0.004850) with: {'n_estimators': 275, 'max_depth': 1, 'min_child_weight': 1}
0.937768 (0.004741) with: {'n_estimators': 306, 'max_depth': 1, 'min_child_weight': 1}
0.938020 (0.004613) with: {'n_estimators': 337, 'max_depth': 1, 'min_child_weight': 1}
0.938225 (0.004487) with: {'n_estimators': 368, 'max_depth': 1, 'min_child_weight': 1}
0.938415 (0.004381) with: {'n_estimators': 400, 'max_depth': 1, 'min_child_weight': 1}
0.934320 (0.005450) with: {'n_estimators': 120, 

NameError: name 'numpy' is not defined

In [ ]:
# choose optimal parameters for random forest

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
max_depth = np.linspace(1,5,5,dtype=int)
n_estimators = np.linspace(120,400,10,dtype=int)
criterion_range=('gini', 'entropy')
kfold = StratifiedKFold(label_encoded_y, n_folds=5, shuffle=True, random_state=42)

In [49]:
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion_range)
grid_search_les = GridSearchCV(RandomForestClassifier(), param_grid=param_grid,  scoring="roc_auc", n_jobs=-1, cv=kfold, verbose=1)
result_les = grid_search_les.fit(X_train, label_encoded_y)

print("Best: %f using %s" % (result_les.best_score_, result_les.best_params_))
means, stdevs = [], []
for params, mean_score, scores in result_les.grid_scores_:
    stdev = scores.std()
    means.append(mean_score)
    stdevs.append(stdev)
    print("%f (%f) with: %r" % (mean_score, stdev, params))


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 13.5min finished


Best: 0.925626 using {'n_estimators': 337, 'max_depth': 5, 'criterion': 'entropy'}
0.894857 (0.020125) with: {'n_estimators': 120, 'max_depth': 1, 'criterion': 'gini'}
0.901226 (0.007952) with: {'n_estimators': 151, 'max_depth': 1, 'criterion': 'gini'}
0.901023 (0.008024) with: {'n_estimators': 182, 'max_depth': 1, 'criterion': 'gini'}
0.897070 (0.009983) with: {'n_estimators': 213, 'max_depth': 1, 'criterion': 'gini'}
0.901378 (0.006663) with: {'n_estimators': 244, 'max_depth': 1, 'criterion': 'gini'}
0.898571 (0.006477) with: {'n_estimators': 275, 'max_depth': 1, 'criterion': 'gini'}
0.901307 (0.008312) with: {'n_estimators': 306, 'max_depth': 1, 'criterion': 'gini'}
0.901229 (0.008411) with: {'n_estimators': 337, 'max_depth': 1, 'criterion': 'gini'}
0.899768 (0.010136) with: {'n_estimators': 368, 'max_depth': 1, 'criterion': 'gini'}
0.902598 (0.007175) with: {'n_estimators': 400, 'max_depth': 1, 'criterion': 'gini'}
0.907777 (0.007032) with: {'n_estimators': 120, 'max_depth': 2, 'cr

In [ ]:
# estimate with train data

In [50]:
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

In [51]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X_train, y_train, test_size=0.2, random_state= 42)

In [ ]:
# evaluate xgboost

In [71]:
y_pred_train = grid_search.predict(X_train1)
roc_auc_score(y_pred_train, y_train1)

0.86667173498362993

In [ ]:
# evaluate random forest

In [53]:
from sklearn.metrics import roc_auc_score

In [54]:
y_pred_les_train = grid_search_les.predict(X_train1)
roc_auc_score(y_pred_les_train, y_train1)

0.86157949950326151

In [ ]:
#upload test data

In [56]:
X_test = pd.read_csv('test_data.csv')

In [57]:
X_test.shape

(13593, 21)

In [76]:
X_test.drop(['duration'], axis=1, inplace=True)

In [58]:
X_test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [60]:
X_test_categorial = pd.get_dummies(X_test[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']])
X_test_categorial.head(3)

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0


In [61]:
X_test.drop(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome'], axis=1, inplace=True)

In [62]:
X_test = pd.concat((X_test, X_test_categorial), axis=1)
X_test = pd.DataFrame(X_test)
print (X_test.shape)

(13593, 62)


In [63]:
X_test.insert(list(X_train.columns).index('default_yes'), 'default_yes', 0)

In [ ]:
#predict test data with random forest

In [64]:
y_pred_test_prob = grid_search_les.predict_proba(X_test)
y_pred_test_prob

array([[ 0.95936961,  0.04063039],
       [ 0.77888935,  0.22111065],
       [ 0.92773822,  0.07226178],
       ..., 
       [ 0.97392186,  0.02607814],
       [ 0.88837918,  0.11162082],
       [ 0.65704749,  0.34295251]])

In [66]:
y_pred_test_prob2=y_pred_test_prob[:,1]
y_pred_test_prob2

array([ 0.04063039,  0.22111065,  0.07226178, ...,  0.02607814,
        0.11162082,  0.34295251])

In [67]:
data_test = pd.DataFrame(data=y_pred_test_prob2, columns=['Prediction'])

In [68]:
data_test.to_csv("C:/Users/Zelich/kaggle_rand_pred.csv",index=True, index_label='Id')

In [ ]:
#predict test data with xgboost

In [72]:
y_pred_test_prob1 = grid_search.predict_proba(X_test)
y_pred_test_prob1

array([[  9.99143481e-01,   8.56536150e-04],
       [  2.46372640e-01,   7.53627360e-01],
       [  9.47620213e-01,   5.23798056e-02],
       ..., 
       [  9.99916911e-01,   8.31068828e-05],
       [  9.67691541e-01,   3.23084518e-02],
       [  8.22794199e-01,   1.77205786e-01]], dtype=float32)

In [73]:
y_pred_test_prob3=y_pred_test_prob1[:,1]
y_pred_test_prob3

array([  8.56536150e-04,   7.53627360e-01,   5.23798056e-02, ...,
         8.31068828e-05,   3.23084518e-02,   1.77205786e-01], dtype=float32)

In [74]:
data_test = pd.DataFrame(data=y_pred_test_prob3, columns=['Prediction'])

In [75]:
data_test.to_csv("C:/Users/Zelich/kaggle_xgboost_pred.csv",index=True, index_label='Id')